In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Config

#path to file generated by other script will take more than one data set
DATA_FILES = [r"./data/synthetic_data_independent_failures_5.csv"] 

SEQUENCE_LENGTH = 24 * 7 * 2 
STEP_SIZE = 24 * 2
FORECAST_HORIZON = 5
BATCH_SIZE = 32
NUM_EPOCHS = 10

# Set device for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


In [55]:
# Data processing 
def load_and_combine_data(file_paths):
    """
    Loads multiple CSV files, finds the union of all sensor features, 
    reindexes dataframes to match the full feature set (filling missing sensors with 0.0), 
    and concatenates them into a single DataFrame.
    """
    all_data = []
    all_sensor_features = set()
    
    # Load data and collect all unique sensor feature names
    non_sensor_cols = ['machine_id', "timestamp",'failure_mode', 'is_precursor_period', 'is_final_failure']
    
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path)
            
            # Identify potential sensor features in the current file
            current_sensor_features = [col for col in df.columns if col not in non_sensor_cols]
            print(current_sensor_features)
            all_sensor_features.update(current_sensor_features)
            
            all_data.append(df)
            print(f"Loaded {file_path} with {len(current_sensor_features)} sensor features.")
        except FileNotFoundError:
            print(f"Warning: File '{file_path}' not found. Skipping.")
    
    if not all_data:
        raise FileNotFoundError("No valid data files were loaded.")

    
    sensor_feature_list = sorted(list(all_sensor_features))
    
    
    final_combined_df = pd.DataFrame()
    
    
    full_column_list = non_sensor_cols + sensor_feature_list
    
    for df in all_data:
        
        df_reindexed = df.reindex(columns=full_column_list, fill_value=0.0)
        
        final_combined_df = pd.concat([final_combined_df, df_reindexed], ignore_index=True)

    print(f"\nSuccessfully combined {len(all_data)} files.")
    print(f"Total rows in combined data: {len(final_combined_df)}")
    print(f"Total unique sensor features used: {len(sensor_feature_list)}")
    
    
    return final_combined_df, sensor_feature_list

In [56]:
try:
    data, sensor_features = load_and_combine_data(DATA_FILES)
except FileNotFoundError as e:
    print(f"Fatal Error: {e}")

['T_internal_sensor', 'V_sensor']
Loaded ./data/synthetic_data_independent_failures_5.csv with 2 sensor features.

Successfully combined 1 files.
Total rows in combined data: 131400
Total unique sensor features used: 2


In [57]:
def create_sequences(data, seq_length, forecast_horizon, step_size):
    sequences = []
    target = []
    for i in range(0, len(data) - seq_length - forecast_horizon + 1, step_size):
        sequences.append(data[i:i+seq_length])
        target.append(data[i+seq_length: i+seq_length+forecast_horizon])
    return np.array(sequences), np.array(target)

In [58]:
def prepare_data(df, sensor_features):
    """Loads, cleans, labels, and scales the data."""
    sensor_data = df[sensor_features].values
   
    # 2. Standardization
    scaler = StandardScaler()
    sensor_data_scaled = scaler.fit_transform(sensor_data)
    
    # 3. Create Sequences
    X_seq, Y_seq = create_sequences(sensor_data_scaled, SEQUENCE_LENGTH, FORECAST_HORIZON, STEP_SIZE)
    
    print(f"\n--- Data Preparation Complete ---")
    print(f"Total time points in raw data: {len(df)}")
    print(f"Total sequences created: {len(X_seq)}")
    print(f"Sequence shape (num_samples, time steps, features): {X_seq.shape}")
    
    return X_seq, Y_seq

In [59]:
X_seq, Y_seq = prepare_data(data, sensor_features)

# Split data into training and testing sets for 3D arrays
indices = np.arange(X_seq.shape[0])
train_indices, test_indices = train_test_split(
    indices, test_size=0.2, random_state=42, shuffle=False)

X_train, X_test = X_seq[train_indices], X_seq[test_indices]
Y_train, Y_test = Y_seq[train_indices], Y_seq[test_indices]


--- Data Preparation Complete ---
Total time points in raw data: 131400
Total sequences created: 2731
Sequence shape (num_samples, time steps, features): (2731, 336, 2)


In [60]:
# PYTORCH Model for fun or more like testing
class TimeDataset(Dataset):
    """Custom PyTorch Dataset for time-series sequences."""
    def __init__(self, X_data, y_data, feature_idx):
        self.X_data = torch.tensor(X_data[:, :, feature_idx: feature_idx + 1], dtype=torch.float32).permute(0, 2, 1)
        self.y_data = torch.tensor(y_data[:, :, feature_idx: feature_idx + 1], dtype=torch.float32).squeeze(-1)
        
    def __len__(self):
        return len(self.y_data)
    
    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [61]:
# # Imbalanced Data
# train_labels_series = pd.Series(Y_train)
# class_counts = train_labels_series.value_counts().sort_index()
# total_samples = len(Y_train)


# class_weights = total_samples / (NUM_CLASSES * class_counts)


# class_weights_tensor = torch.tensor(class_weights.values, dtype=torch.float32).to(DEVICE)

# print("\nCalculated Class Weights (Higher = More Important):")
# for i, weight in enumerate(class_weights_tensor):
#     print(f"Class {i} Weight: {weight:.2f}")

In [62]:
X_train.shape, Y_train.shape

((2184, 336, 2), (2184, 5, 2))

In [63]:
# Create PyTorch Datasets and DataLoaders
train_loader = []
test_loader = []

for i in range(X_train.shape[2]):
    train_dataset = TimeDataset(X_train, Y_train, i)
    test_dataset = TimeDataset(X_test, Y_test, i)

    train_loader.append(DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False))
    test_loader.append(DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False))

In [64]:
class CNNClassifier(nn.Module):
    """
    Simple 1D CNN for time-series regression built with PyTorch.
    Matches the Keras Conv1D + MaxPool + Dense architecture.
    """
    def __init__(self, seq_length, forecast_horizon):
        super(CNNClassifier, self).__init__()
        
        # Layer 1: Conv1D (64 filters, kernel size 3)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=0)
        
        # Layer 2: Max Pooling
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Calculate flattened size after conv and pooling
        # seq_length -> (seq_length - 2) after conv -> (seq_length - 2) // 2 after pool
        conv_output_size = (seq_length - 2) // 2
        flattened_size = 64 * conv_output_size
        
        # Layer 3: Fully connected (Dense) layer with 100 units
        self.fc1 = nn.Linear(flattened_size, 100)
        
        # Layer 4: Output layer for regression
        self.fc2 = nn.Linear(100, forecast_horizon)

    def forward(self, x):
        # Conv + ReLU
        x = torch.relu(self.conv1(x))
        
        # Max Pooling
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Dense layer + ReLU
        x = torch.relu(self.fc1(x))
        
        # Output layer (no activation for regression)
        x = self.fc2(x)
        
        return x

In [65]:
# --- 3. TRAINING AND EVALUATION ---

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    """PyTorch training loop."""
    model.train()
    print(f"\n--- Starting PyTorch Model Training ({num_epochs} Epochs) ---")
    
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [66]:
# Build the PyTorch CNN model
model = CNNClassifier(X_train.shape[1], FORECAST_HORIZON).to(DEVICE)

print("\nModel Architecture:")
print(model)


Model Architecture:
CNNClassifier(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=10688, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)


In [67]:
# Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader[0], criterion, optimizer, NUM_EPOCHS)


--- Starting PyTorch Model Training (10 Epochs) ---
Epoch [1/10], Loss: 0.0379
Epoch [2/10], Loss: 0.0214
Epoch [3/10], Loss: 0.0232
Epoch [4/10], Loss: 0.0245
Epoch [5/10], Loss: 0.0169
Epoch [6/10], Loss: 0.0239
Epoch [7/10], Loss: 0.0155
Epoch [8/10], Loss: 0.0171
Epoch [9/10], Loss: 0.0161
Epoch [10/10], Loss: 0.0156


In [78]:
def evaluate_model(model, test_loader):
    """PyTorch evaluation loop for regression."""
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            
            # Forward pass - get predictions
            outputs = model(inputs)
            
            # Collect predictions and targets (no argmax for regression)
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())
            
    return np.array(y_true), np.array(y_pred)

def calculate_msre(y_pred, y_true):
    avg_error_per_column = np.mean(np.abs(y_true - y_pred), axis=0)
    print(f"Average error per column: {avg_error_per_column}")
    mse = np.mean((y_true - y_pred) ** 2)
    print(f"Total MSE: {mse}")
    return np.sqrt(mse)

In [ ]:
# Evaluate the model
print("\n--- Evaluating Model Performance ---")
y_true, y_pred = evaluate_model(model, test_loader[0])

calculate_msre(y_pred, y_true)

# # Basic classification report
# print("\nClassification Report (PyTorch):")
# report = classification_report(y_true, y_pred, 
#                                 target_names=['Normal (0)', 'Precursor (1)', 'Failure (2)'],
#                                 labels=[0, 1, 2])
# print(report)

# print("\nConfusion Matrix:")
# print(confusion_matrix(y_true, y_pred))


--- Evaluating Model Performance ---
Average error per column: [0.11688787 0.11731971 0.12023129 0.12289765 0.12906395]
Total MSE: 0.02321123518049717


np.float32(0.15235233)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Regression metrics
mse = mean_squared_error(y_true.flatten(), y_pred.flatten())
mae = mean_absolute_error(y_true.flatten(), y_pred.flatten())
rmse = np.sqrt(mse)
r2 = r2_score(y_true.flatten(), y_pred.flatten())

print("\nRegression Metrics:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")


Regression Metrics:
Mean Squared Error (MSE): 0.0232
Root Mean Squared Error (RMSE): 0.1524
Mean Absolute Error (MAE): 0.1213
R² Score: 0.9373
